In [51]:
from diagrams import Diagram, Cluster, Edge
from diagrams.onprem.workflow import Airflow
from diagrams.aws.storage import S3
from diagrams.programming.language import Python
from diagrams.custom import Custom

# Define the visual context
graph_attr = {
    "fontsize": "10",       # Font size for the text
    "size": "12,8",         # Diagram width and height (inches)
    "nodesep": "1.0",       # Increased spacing between nodes
    "ranksep": "1.0",       # Increased vertical spacing between levels
    "dpi": "250"            # Dots per inch (increase this to improve resolution)
}

filename = "flow_diagram"

with Diagram("", show=False, direction="LR", graph_attr=graph_attr, filename=filename):
    
    # Data Ingestion with Airflow (Top Cluster)
    with Cluster("Data Ingestion Pipeline"):
        airflow = Airflow("Airflow Trigger")
        scraper = Python("Scraper")
        storage_s3 = S3("S3: PDF & Images")
        data_source = Custom("CFA Institute \n Research Publications", "./images/data_source.png")
        structured_data_snowflake = Custom("Snowflake: Structured \nData", "./images/snowflake.png")

        # Workflow connections in the Data Ingestion Pipeline
        data_source >> Edge(label="Web Scraping", minlen="2") >> airflow
        airflow >> Edge(label="Trigger Scraping", minlen="2") >> scraper
        scraper >> Edge(label="Store PDFs & Images", minlen="2") >> storage_s3  
        scraper >> Edge(label="Store Structured Data", minlen="2") >> structured_data_snowflake  
        storage_s3 >> Edge(label="Pdf/Image Links", style='curved') >> structured_data_snowflake

    # Client-Facing Application (Bottom Cluster)
    with Cluster("Client-Facing Application", graph_attr={"pad": "0.5"}):  # Reduced padding for compactness
        fastapi_service = Custom("FastAPI", "./images/FastAPI.png")
        nvidia_nemo = Custom("NVIDIA NeMo (NIM)", "./images/nvidia.png")
        streamlit_app = Custom("Streamlit UI", "./images/Streamlit.png")
        
        pinecone_service = Custom("Pinecone", "./images/pinecone.png") 
        langchain_service = Custom("LangChain", "./images/langchain.png")
        
        # Connections based on new requirements
        
        # FastAPI is only connected with NVIDIA NeMo and Streamlit
        fastapi_service >> Edge(label="Generate Summaries, Q/A", minlen="1") >> nvidia_nemo
        fastapi_service >> Edge(label="Serve Data", minlen="1") >> streamlit_app
        
        # Streamlit is connected to FastAPI, Pinecone, and LangChain
        streamlit_app >> Edge(label="Query Pinecone", minlen="1") >> pinecone_service
        streamlit_app >> Edge(label="Use LangChain for Orchestration", minlen="1") >> langchain_service
        
        # NVIDIA NeMo is connected to FastAPI and LangChain
        nvidia_nemo >> Edge(label="Process and Respond", minlen="1") >> fastapi_service
        nvidia_nemo >> Edge(label="Interact with LangChain", minlen="1") >> langchain_service
        
        # Pinecone is connected to Streamlit and NVIDIA NeMo
        pinecone_service >> Edge(label="Return Results to Streamlit", minlen="1") >> streamlit_app
        pinecone_service >> Edge(label="Interact with NVIDIA NeMo", minlen="1") >> nvidia_nemo
        
        # LangChain is connected to Streamlit and NVIDIA NeMo
        langchain_service >> Edge(label="Return Processed Results to Streamlit", minlen="1") >> streamlit_app
        langchain_service >> Edge(label="Process with NVIDIA NeMo", minlen="1") >> nvidia_nemo

        # New connections:
        
        # S3 connected to FastAPI (Provide Access to Files)
        storage_s3 >> Edge(label="Provide Access to Files", minlen="2", style='curved') >> fastapi_service
        
        # Snowflake connected to FastAPI (Provide Structured Data Access)
        structured_data_snowflake >> Edge(label="Provide Structured Data Access", minlen="", style='curved') >> fastapi_service